In [ ]:
# It prepares the training data.
import math
def normalformat(datafile):
    with open(datafile) as f: # open file and read it in
        lines = f.readlines()

    data_0 = lines[0] # prepare the list for the data
    
    # separate the data from the texts
    data_0 = data_0[2:(len(data_0)-2)]
    data_1 = data_0.split("NormalizedLandmark(")
    data_1 = data_1[1:22]
    
    # unify the data
    for i in range(19):
        data_1[i] = data_1[i][0:len(data_1[i])-3]
    data_1[20] = data_1[20][0:len(data_1[20])-1]
    
    # remove "="
    data_list = []
    for i in range(len(data_1)):
        list = data_1[i].split("=")
        data_list.append(list)
        
    # add the 21 point's coordinates
    data = []
    for i in data_list:
        list_0 = []
        list_0.append(float(i[1][0:-3]))
        list_0.append(float(i[2][0:-3]))
        list_0.append(float(i[3][0:-12]))
        data.append(list_0)
        
    # choose reference point
    ref_point = data[0]
    
    # distance from every point to every point
    difference = []
    
    for i in range(21): # because there are 21 points
        for n in range(len(data)-i):
            # x coordinate
            x = data[i][0] - data[n+i][0]
    
            # y coordinate
            y = data[i][1] - data[n+i][1]
    
            # z coordinate
            z = data[i][2] - data[n+i][2]
        
            difference.append([x, y, z])

    distance = []
    
    # see which point is lower (y < 0 == up, y >= 0 == down)
    for i in difference:
        if i[2] < 0:
            multiplier = -1
        else:
            multiplier = 1
            
        dist = math.sqrt(i[0]**2 + i[1]**2 + i[2]**2)*multiplier # multiply with -1 if the first point is lower
        distance.append(dist)
    
    # standardize the data by the distance of point 0 and 16
    unit = 1/(distance[16])
    
    # add training data to the list if data != 0
    training_data = []
    for i in distance:
        if i != -0.0:
            training_data.append(i*unit)
    
    return training_data

In [ ]:
# Create dataframe (training data) from txt files.
import pandas as pd
import os

# Up
# create dictionary
data = {}
directory = 'Up'

# read in files from directory
index = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    
    # try to add the hand landmarks to the dictionary 
    try:
        # the hand type is up
        list_0 = ['up']
        
        # add the hand landmark coordinates to the list
        for i in normalformat(f):
            list_0.append(i)
        
        # load the list to the dictionary {sequence of indexes: list}
        data[str(index)] = list_0
        
        # add 1 to the sequence
        index += 1
        
    except:
        pass

# Down
directory = 'Down'
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    try:
        list_0 = ['down']
        for i in normalformat(f):
            list_0.append(i)
        data[str(index)] = list_0
        index += 1
    except:
        pass

# Click
directory = 'Click'
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    try:
        list_0 = ['click']
        for i in normalformat(f):
            list_0.append(i)
        data[str(index)] = list_0
        index += 1
    except:
        pass

# Mouse
directory = 'Mouse'
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    try:
        list_0 = ['mouse']
        for i in normalformat(f):
            list_0.append(i)
        data[str(index)] = list_0
        index += 1
    except:
        pass

# Stay
directory = 'Stay_clicked'
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    try:
        list_0 = ['stay']
        for i in normalformat(f):
            list_0.append(i)
        data[str(index)] = list_0
        index += 1
    except:
        pass
    
# create dataframe
columns = ('hand', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210')
df = pd.DataFrame(data).T
df.columns = columns

In [ ]:
# Separate the training and testing data.
from sklearn.model_selection import train_test_split

X = df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210']].values
y = df['hand'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
# Train logistic regression model and test.
from sklearn.linear_model import LogisticRegression

# train model
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

# test model
predictions = logmodel.predict(X_test)

# show results
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
# Train decision tree model and test.
from sklearn.tree import DecisionTreeClassifier

# train model
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

# test model
predictions = dtree.predict(X_test)

# show results
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.datasets import load_iris
import pickle

# load logistic regression model to a pickle file and upload (because logistic regression trained to 99% and decision tree trained to 96%)
filename = "2024_stemsisters_project.pickle"
pickle.dump(logmodel, open(filename, "wb"))
loaded_model = pickle.load(open(filename, "rb"))